In [15]:
import yfinance as yf
import pandas as pd
import numpy as np
# Define the list of indices and equities
indices = ['^GSPC', '^FTSE', '^N225', '^GDAXI', '^FCHI']
equities = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'TM']

# Load the daily data for indices
index_data = yf.download(indices, start='2010-01-01')['Adj Close']

# Create an empty DataFrame to store equity data
equity_data = pd.DataFrame()

# Retrieve data for each equity
for equity in equities:
    equity_ticker = yf.Ticker(equity)
    equity_history = equity_ticker.history(start='2010-01-01')
    equity_data[equity] = equity_history['Close']

# Convert equity_data to time zone-naive
equity_data = equity_data.tz_localize(None)

# Combine index and equity data
data = pd.concat([index_data, equity_data], axis=1)

# Calculate daily returns
returns = data.pct_change().dropna()

# Calculate cumulative returns
cumulative_returns = (1 + returns).cumprod()

# Calculate maximum drawdowns
roll_max = cumulative_returns.rolling(window=len(cumulative_returns), min_periods=1).max()
daily_drawdown = cumulative_returns / roll_max - 1.0
max_drawdown = daily_drawdown.min()

# Calculate volatility
volatility = returns.std() * np.sqrt(252)

# Calculate the Sharpe ratio
risk_free_rate = 0.0  # Assuming no risk-free rate for simplicity
daily_risk_free_rate = (1 + risk_free_rate) ** (1/252) - 1
excess_returns = returns - daily_risk_free_rate
sharpe_ratio = np.mean(excess_returns) / np.std(returns) * np.sqrt(252)

# Calculate the Sortino ratio
downside_returns = returns.copy()
downside_returns[downside_returns > 0] = 0
sortino_ratio = np.mean(excess_returns) / np.std(downside_returns) * np.sqrt(252)

# Create tables for indices and equities
index_metrics = pd.DataFrame({'Cumulative Returns': cumulative_returns.iloc[-1],
                              'Volatility': volatility,
                              'Max Drawdown': max_drawdown,
                              'Sharpe Ratio': sharpe_ratio,
                              'Sortino Ratio': sortino_ratio},
                             index=indices)

equity_metrics = pd.DataFrame({'Cumulative Returns': cumulative_returns.iloc[-1, len(indices):],
                               'Volatility': volatility[len(indices):],
                               'Max Drawdown': max_drawdown[len(indices):],
                               'Sharpe Ratio': sharpe_ratio[len(indices):],
                               'Sortino Ratio': sortino_ratio[len(indices):]},
                              index=equities)

# Display the tables
print("Index Metrics:")
print(index_metrics)
print("\nEquity Metrics:")
print(equity_metrics)


[*********************100%***********************]  5 of 5 completed
Index Metrics:
        Cumulative Returns  Volatility  Max Drawdown  Sharpe Ratio   
^GSPC             3.699927    0.174476     -0.339250      0.813129  \
^FTSE             1.410269    0.159814     -0.366055      0.887729   
^N225             2.891502    0.201641     -0.317989      0.703585   
^GDAXI            2.690902    0.199732     -0.387794      0.710311   
^FCHI             1.866471    0.201984     -0.385585      0.702391   

        Sortino Ratio  
^GSPC        1.252372  
^FTSE        1.376468  
^N225        1.121879  
^GDAXI       1.120382  
^FCHI        1.105514  

Equity Metrics:
       Cumulative Returns  Volatility  Max Drawdown  Sharpe Ratio   
AAPL            26.963060    0.281026     -0.437972      0.504835  \
MSFT            13.504792    0.256669     -0.371485      0.552742   
AMZN            17.363704    0.326132     -0.561453      0.435014   
GOOGL            7.826869    0.268609     -0.443201      0

In [14]:
index_metrics.to_csv('./Index.csv')
equity_metrics.to_csv('./Equity.csv')